In [86]:
import pandas as pd
import sys
import numpy as np
import json
from utils import fen_to_vector


In [87]:
def extract_cp(eval):
    first_pv = eval[0]["pvs"]
    try:
        return first_pv[0]["cp"]
    except KeyError:
        return first_pv[0]["mate"] #TODO: Make real big number pos or neg

    
def extract_depth(eval):
    depth = eval[0]["depth"]
    return depth

def extract_fen(eval):
    return eval

In [90]:
# Fixed this function, uncomment df_depth if we need depth values, otherwise don't
def json_to_dataframe(path, num_chunks, chunksize=1000):
  reader = pd.read_json(path, lines=True, chunksize=chunksize)
  df_fen = pd.DataFrame(columns=["fen"])
  df_cp = pd.DataFrame()
  #df_depth = pd.DataFrame()
  
  i = 0
  for chunk in reader:
      # concatenate all the chunks together into one column
      df_fen = pd.concat([df_fen, chunk["fen"]], axis=0)
      df_cp = pd.concat([df_cp, chunk["evals"].apply(extract_cp)], axis=0)
      #df_depth = pd.concat([df_depth, chunk["evals"].apply(extract_depth)], axis=0)
      i+=1
      print(f"Loading data into dataframe {(i/num_chunks * 100):.2f}% done.")
      sys.stdout.flush()
      
      if i == num_chunks:
          break
          
  df_cp = df_cp.rename(columns={"evals":"cp"})
  #df_depth = df_depth.rename(columns={"evals":"depth"})
  df = pd.concat([df_fen, df_cp], axis=1)
  return df

# get one chunk of 10000 data examples
num_chunks = 100
chunksize = 10000
df = json_to_dataframe("lichess_db_eval.jsonl", num_chunks, chunksize)


Loading data into dataframe 1.00% done.
Loading data into dataframe 2.00% done.
Loading data into dataframe 3.00% done.
Loading data into dataframe 4.00% done.
Loading data into dataframe 5.00% done.
Loading data into dataframe 6.00% done.
Loading data into dataframe 7.00% done.
Loading data into dataframe 8.00% done.
Loading data into dataframe 9.00% done.
Loading data into dataframe 10.00% done.
Loading data into dataframe 11.00% done.
Loading data into dataframe 12.00% done.
Loading data into dataframe 13.00% done.
Loading data into dataframe 14.00% done.
Loading data into dataframe 15.00% done.
Loading data into dataframe 16.00% done.
Loading data into dataframe 17.00% done.
Loading data into dataframe 18.00% done.
Loading data into dataframe 19.00% done.
Loading data into dataframe 20.00% done.
Loading data into dataframe 21.00% done.
Loading data into dataframe 22.00% done.
Loading data into dataframe 23.00% done.
Loading data into dataframe 24.00% done.
Loading data into datafra

In [91]:
df.to_csv(f"data.csv", encoding='utf-8', index=False)